<a href="https://colab.research.google.com/github/nehathipperudrappa/MAJOR_PROJECT/blob/main/Majorproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install -q wordcloud
import wordcloud
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv('/content/Review_data.csv')
df

In [ ]:
df['rating'] = df['rating'].replace(5,'Good')
df['rating'] = df['rating'].replace(4,'Good')
df['rating'] = df['rating'].replace(3,'Good')
df['rating'] = df['rating'].replace(2,'Bad')
df['rating'] = df['rating'].replace(1,'Bad')

In [ ]:
#preprocess data
df['review'] = df['review'].str.lower()  #Convert to lower case
df['review'] = df['review'].str.replace('\d+', '') #removing numbers
df['review'] = df['review'].str.replace('.','')
df['review'] = df['review'].str.replace('-','')
df['review'] = df['review'].str.replace('?','')
df['review'] = df['review'].str.replace("'",'')
df['review'] = df['review'].str.replace('  ','')  #for extra spaces


In [ ]:
#tokenization and stemming
from nltk.stem.snowball import SnowballStemmer
df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row['review']), axis=1)
stemmer = SnowballStemmer("english")
df['stemmed'] = df['tokenized_sents'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
df = df.drop(columns=['tokenized_sents'])
df['review'] = df['stemmed'].apply(lambda x: ' '.join(map(str, x)))
df = df.drop(columns=['stemmed'])

In [ ]:
x = df.iloc[:,0]
y = df.iloc[:,1]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0,stratify=y)

In [ ]:
#Vectorization and model fitting using pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
text_model = Pipeline([('tfidi',TfidfVectorizer()),('model',SVC())])
text_model.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
y_pred = text_model.predict(x_test)
accuracy_score(y_test,y_pred)

In [ ]:
#confusion matrix
confusion_matrix(y_test,y_pred)

In [ ]:
#Classification report
classification_report(y_test,y_pred)

In [ ]:
import joblib
joblib.dump(text_model,'Email_Class')

In [ ]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

In [ ]:
%%writefile app.py
import streamlit as st
import sklearn
import joblib
model =joblib.load('Email_Class')

add_selectbox = st.sidebar.radio("Our page",['Home',"Rate us"])
if(add_selectbox == 'Home'):
  st.title("Review classifier")
  st.image('https://static.businessworld.in/article/article_extra_large_image/1600858456_HulSrV_Flipkart.jpg')

  ip = st.text_input('Enter your review')
  p = model.predict([ip])
  if st.button('Predict'):
    st.header(op[0])
    if(op[0]=='Good'):
      st.markdown(""":smile:""")
    else:
      st.markdown(""":angry:""")
else:
  st.title("Rate US")
  st.slider("Rating",0,5)
  if st.button("Submit"):
    st.header("Thanks for the rating")
  
  

Overwriting app.py


In [ ]:
!nohup streamlit run app.py&
url = ngrok.connect(port = '8501')
url

In [ ]:
!pip install pipreqs

In [ ]:
!pipreqs /content

INFO: Successfully saved requirements file in /content/requirements.txt
